# DuckLake Tutorial

## Overview

DuckLake is a DuckDB extension implementing an Iceberg-compatible lakehouse with ACID transactions, time travel queries, and schema evolution. It combines DuckDB's columnar analytics with PostgreSQL catalog management.

**Key capabilities:**
- ACID transactions on data lake operations
- Snapshot-based time travel
- Schema evolution without data migration
- Parquet-based columnar storage

# Quick Start

## Start Required Services

### Using taskipy (Recommended)
```bash
# Start all services
uv run task start

# Check status
uv run task status

# Stop all services
uv run task stop
```

### Service Management
```bash
# Individual services
uv run task start-postgres
uv run task start-minio
uv run task stop-postgres
uv run task stop-minio

# Monitoring
uv run task logs-postgres    # PostgreSQL logs only
uv run task logs-minio       # MinIO logs only
uv run task logs             # Both services (prefixed)

# Development workflow
uv run task dev  # Start services + notebook

# Data management
uv run task clean      # Stop services, remove volumes
uv run task reset      # Clean + restart
uv run task reset-data # Clear DuckLake data only
```

### Access Points
- **MinIO Console**: http://localhost:${MINIO_CONSOLE_PORT:-9001} (${MINIO_USER:-minioadmin}/${MINIO_PASSWORD:-minioadmin})
- **PostgreSQL**: localhost:${POSTGRES_PORT:-5432} (${POSTGRES_USER:-ducklake}/${POSTGRES_PASSWORD:-ducklake123})

In [ ]:
import logging
import os
import shutil
import subprocess
import time

import click
import duckdb
import numpy as np
import pandas as pd
from click import Context
from rich import box
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel
from rich.progress import (
    BarColumn,
    Progress,
    SpinnerColumn,
    TextColumn,
    TimeElapsedColumn,
)
from rich.syntax import Syntax
from rich.table import Table
from helpers import *

# Initialize rich console and logger
console = Console()

setup_logging()

console.print("✓ [green]Imports successful[/green]")

In [ ]:

reset_ducklake_data()

check_postgresql()
conn = check_duckdb()

if conn is not None:
    conn = setup_extensions(conn)
    pg_config = test_postgresql_connection()

    if conn is not None and pg_config is not None:
        db_name = initialize_ducklake(conn, pg_config)

In [ ]:
if conn is not None:
    create_sample_data(conn)


In [ ]:
if conn is not None:
    demonstrate_queries(conn)


# Phase 2: ACID & Time Travel

## ACID Transaction Test

```python
conn.begin()
conn.execute("INSERT INTO customers VALUES (101, 'Test', 'test@example.com', '2024-01-01', 'Test City', 30)")
conn.execute("INSERT INTO sales VALUES (501, 101, 'Product', 999.99, '2024-01-01 10:00:00', 'Test')")
conn.rollback()  # Atomicity: all changes undone
```

## Snapshots

Each data modification creates an immutable snapshot. Snapshots enable time travel queries to historical states.

**Snapshot lifecycle**:
1. Initial state (snapshot 1)
2. Add customers (snapshot 2)
3. Add sales (snapshot 3)
4. Update ages (snapshot 4)
5. Add column (snapshot 5)

In [ ]:
if conn is not None:
    demonstrate_acid_transactions(conn)

In [ ]:
if conn is not None and db_name is not None:
    demonstrate_time_travel(conn, db_name)

## Time Travel

Query historical data using snapshot versions:

```sql
-- Current data
SELECT customer_id, age FROM customers WHERE customer_id <= 5;

-- Historical data
SELECT customer_id, age FROM customers 
FOR SYSTEM_TIME AS OF SNAPSHOT 5
WHERE customer_id <= 5;
```

**Use cases**: Audit trails, debugging, compliance, A/B testing
**Implementation**: Snapshot-based versioning with immutable storage

In [ ]:
if conn is not None:
    demonstrate_schema_evolution(conn)

## Schema Evolution

Add columns without data migration:

```sql
-- Add new column
ALTER TABLE customers ADD COLUMN loyalty_points INTEGER DEFAULT 0;

-- Update with calculated values
UPDATE customers 
SET loyalty_points = age * 10
WHERE loyalty_points IS NULL;
```

**Benefits**: 
- Non-destructive changes
- Backward compatibility
- Zero downtime
- Automatic default handling

# Phase 3: Performance Analysis

## Query Performance

Measure execution time across query complexity levels:
- Simple aggregation: `SELECT COUNT(*) FROM customers`
- Join aggregation: Customer sales by city
- Complex analytics: Multi-dimensional aggregation with filtering

## Storage Efficiency

**Parquet advantages**:
- Columnar layout for analytical queries
- Built-in compression (typically 80-90% reduction)
- Predicate pushdown for selective reads
- Cross-platform compatibility

**Metrics analyzed**:
- File sizes and compression ratios
- Row counts and column statistics
- Query execution times
- I/O throughput

In [ ]:
if conn is not None:
    demonstrate_performance(conn)

In [ ]:
explore_parquet_files()

In [ ]:
demonstrate_data_compression()

In [ ]:
if conn is not None:
    show_maintenance_info(conn)